# **Hindi Whisper ASR using OpenAI Whisper Model**

using one of the audio file of kathbath dataset (audio hindi) to test the model and find WER

In [ ]:
!pip install pyloudnorm
!pip install pydub

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
%matplotlib inline
import librosa.display
from IPython.display import Audio
import pandas as pd
import os
import glob

# import splitfolders
import skimage.io
import pydub

import librosa as lr


In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5knmnl_p
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5knmnl_p
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import whisper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_tiny = whisper.load_model("tiny") #Tiny whisper model

In [ ]:
model_medium = whisper.load_model("medium") #Medium whisper model

In [81]:
model_large = whisper.load_model("large")

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:65: UserWarning: /root/.cache/whisper/large-v3.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|█████████████████████████████████████| 2.88G/2.88G [00:46<00:00, 65.9MiB/s]


In [ ]:
def Convert_wav_2_mp3(rootdir):

  for it in os.scandir(rootdir):
    if it.is_dir():
      category_dir = it.path
      print(category_dir)

      #Read contents in this directory
      for file in os.listdir(category_dir):
        filename = os.fsencode(file)
        filename = filename.decode()
        #print(filename)
        if filename.endswith(".wav"):
          # Get an audio file to be processed from the directory
           audio_path = (os.path.join(category_dir,filename))
           #print(audio_path)

           mp3_file = os.path.splitext(filename)[0] + '.mp3' #name the file with .mp3 extension
           path_to_save = (os.path.join(category_dir,mp3_file))
           #print(path_to_save)

           sound = pydub.AudioSegment.from_wav(audio_path) #convert to .mp3
           sound.export(path_to_save, format="mp3")

In [64]:
from pydub import AudioSegment
audioPathHindi = '/content/drive/MyDrive/hindi/test/audio/844424930627596-153-f.wav'
outputPathMp3 = '/content/drive/MyDrive/hindi/test/audio/audio1-f.mp3'

sound = AudioSegment.from_wav(audioPathHindi)

sound.export(outputPathMp3, format="mp3")


<_io.BufferedRandom name='/content/drive/MyDrive/hindi/test/audio/audio1-f.mp3'>

In [65]:
audioHindi = whisper.load_audio("/content/drive/MyDrive/hindi/test/audio/audio1-f.mp3")

In [66]:


audioHindi = whisper.pad_or_trim(audioHindi)


In [68]:
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audioHindi).to(model_medium.device)

# detect the spoken language
_, probs = model_medium.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model_medium, mel, options)

# print the recognized text
print(result.text)

Detected language: hi
रिया चक्रवर्ती को आज भाई खला जेल में चटाई बिचाकर राद गुजार नहीं पड़ेगी।



Hindi to English




In [80]:
import jiwer

# Ground truth (actual) text and transcribed text
actual_text = "आज रिया चक्रवर्ती को भाई खाला जेल में चटाई पर रहने से मुक्ति मिल जाएगी."
transcribed_text = result.text

# Normalize texts if needed
# For Hindi, normalization might not be as crucial, but it's still good practice

# Calculate WER
wer = jiwer.wer(actual_text, transcribed_text)
print(f"WER: {wer * 100:.2f} %")


WER: 60.00 %


In [78]:
options = {"fp16": False, "language": "hi", "task": "translate"}
result = model_medium.transcribe(audioHindi, **options)
print(result)

{'text': ' Rhea Chakrabarti will have to spend the night in Bhai Khala Jail', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.0, 'text': ' Rhea Chakrabarti will have to spend the night in Bhai Khala Jail', 'tokens': [50364, 497, 27799, 761, 514, 5305, 40155, 486, 362, 281, 3496, 264, 1818, 294, 13550, 1301, 11681, 5159, 508, 864, 50714], 'temperature': 0.0, 'avg_logprob': -0.44376407970081677, 'compression_ratio': 0.9846153846153847, 'no_speech_prob': 0.07707208395004272}], 'language': 'hi'}
